In [1]:
from IPython.core.display_functions import clear_output
import ipywidgets as widgets
from IPython.display import display

from dashboard_requests import *

In [3]:
def CreateChain():
    with output1:
        %run notebooks/crud/chains/createchain.ipynb
    # response = widgets.HTML(value='')
    # return response

def UpdateChain():
    %run notebooks/crud/chains/updatechain.ipynb
    # response = widgets.HTML(value='')
    # return response

def CreateHotel():
    %run notebooks/crud/hotel/createhotel.ipynb
    # response = widgets.HTML(value='')
    # return response

def UpdateHotel():
    %run notebooks/crud/hotel/updatehotel.ipynb
    # response = widgets.HTML(value='')
    # return response

In [4]:
# ** Switch between log in and sign in input credentials
def change_view1(button):
    with output:
        clear_output()
        if button.description == 'Sign Up':
            button.description = 'Log In'
            login_box.children = [employee_id, first_name, last_name, username, password, sign_up_button]
        else:
            button.description = 'Sign Up'
            login_box.children = [username, password, login_button]
            
def change_view2(button):
    with output:
        clear_output()
        login_box.children = [username, password, login_button]
        toggle_button.description = 'Sign Up'
        toggle_button.layout.visibility = 'visible'
        access.disabled = True
        with output1: clear_output()

In [5]:
# ** Login Credentials
def login(button):
    # ** Login Logic
    with output:
        response = Check_Login(username.value, password.value)
        if response.status_code == 200:  # Check if the request was successful
            clear_output()
            print(f"Login successful.")
            login_box.children = [username, password, logout_button]
            toggle_button.layout.visibility = 'hidden'
            access(response)
        else:
            clear_output()
            print(f"Login failed. Please try again. username: {username.value}, password: {password.value}")

# ** Signup Credentials
def signup(button):
    # ** Signup Logic
    with output:
        employee = Check_Employee(employee_id.value, first_name.value, last_name.value)
        if employee.status_code == 200:
            post_login = Post_Login(employee_id.value, username.value, password.value)
            if post_login.status_code == 201:
                clear_output()
                print(f"Sign up successful, Welcome!")
                login_box.children = [username, password, logout_button]
                toggle_button.layout.visibility = 'hidden'
            elif post_login.status_code == 400:
                clear_output()
                print(f"Employee have account")
            else:
                clear_output()
                print(f"Username exist.")
        else:
            clear_output()
            print(f"Employee does not exist. {employee_id.value}, {first_name.value}, {last_name.value}")

In [6]:
# ** Login Widgets
username = widgets.Text(placeholder="---", description='Username')
password = widgets.Password(placeholder="---", description='Password')
login_button = widgets.Button(description="Log In")
login_button.on_click(login)

# ** Sigin Widgets
employee_id = widgets.Text(placeholder="---", description='Employee ID')
first_name = widgets.Text(placeholder="---", description='First Name')
last_name = widgets.Text(placeholder="---", description='Last Name')
sign_up_button = widgets.Button(description="Sign Up")
sign_up_button.on_click(signup)

logout_button = widgets.Button(description="Log Out")
logout_button.on_click(change_view2)

In [7]:
# ** Initial State
login_box = widgets.VBox([username, password, login_button])

# ** Switch between log in and sign in credentials
toggle_button = widgets.Button(description="Sign Up")
toggle_button.on_click(change_view1)

In [8]:
def access(logins):
    # with output:
        with output1:
            clear_output()
            eid = logins.json()['Login']['eid']
            employee = Get_Employee(eid).json()['Employee']
            fname = employee['fname']
            lname = employee['lname']
            # print(employee)
            welcome = widgets.HTML(f"<p style='font-size:24px;'>Welcome {fname} {lname}!</p>")
            display(welcome)
            position = employee['position']
            # access_box = widgets.VBox([])
            if position == 'Regular':
                pass
            elif position == 'Supervisor':
                pass
            elif position == 'Administrator':
                accessAdministrator()
            #     createchain = CreateChain()
            #     updatechain = UpdateChain()
            #     createhotel = CreateHotel()
            #     updatehotel = UpdateHotel()
            #     access_box.children = [createchain, updatechain, createhotel, updatehotel]
            #     
            # display(access_box)
        

In [2]:
header_text = widgets.HTML("<h1 style='text-align: center; color: white; background-color: #333; width: 100%; margin: 0; "
                           "padding: 15px 0;'>Hotel Analytics Systems by DB Beasts</h1>"
                           "<h1 style='text-align: center; color: white; background-color: #333; width: 100%; margin: 0; "
                           "padding: 15px 0;'>FRONTEND</h1>")

In [9]:
display(header_text)

HTML(value="<h1 style='text-align: center; color: white; background-color: #333; width: 100%; margin: 0; paddi…

In [10]:
output = widgets.Output()
display(output)

Output()

In [11]:
display(login_box)
display(toggle_button)

Button(description='Sign Up', style=ButtonStyle())

In [12]:
# intento = """
# <html>
# <head>
# <title>Font Awesome Icons</title>
# <meta name="viewport" content="width=device-width, initial-scale=1">
# <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
# </head>
# <body>
# 
# <p>Loading:</p>
# <i class="fa fa-spinner fa-spin" style="font-size:24px"></i>
# 
# </body>
# </html> 
# """
# 
# display(HTML(intento))

In [13]:
# output2 = widgets.Output()
# 
# with output2:
#     file = open("fondo.png", "rb")
#     image = file.read()
#     fondo = widgets.Image(value=image, style={'width': '100%', 'height': '100%vh'})
#     display(fondo)
#     
# display(output2)

Output()

In [14]:
output1 = widgets.Output()
display(output1)

Output()

In [15]:
def accessAdministrator():
    CreateChain()
    UpdateChain()
    CreateHotel()
    UpdateHotel()